# Exercício 2


In [ ]:
**INTEGRANTE 01:** Eliel Oliveira RG 1151482-ES RA ex188986
    
**INTEGRANTE 02:** Leander Silveria RG 4857661 RA ex188752

## Sobre a base de dados

Neste laboratório você utilizará um dataset contendo informações sobre filmes e cinemas. 

**Collections:**
* comments
* movies
* sessions
* theaters
* users




Veja um exemplo de documento de um dos filmes da collection `movies` e entenda a estrutura/schema:

```python
{
	'_id': ObjectId('573a1390f29313caabcd4135'),
	'awards': {
		'nominations': 0,
		'text': '1 win.',
		'wins': 1
	},
	'cast': ['Charles Kayser', 'John Ott'],
	'countries': ['USA'],
	'directors': ['William K.L. Dickson'],
	'fullplot': 'A stationary camera looks at a large ...',
	'genres': ['Short'],
	'imdb': {
		'id': 5,
		'rating': 6.2,
		'votes': 1189
	},
	'lastupdated': '2015-08-26 00:03:50.133000000',
	'num_mflix_comments': 1,
	'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
	'rated': 'UNRATED',
	'released': datetime(1893, 5, 9, 0, 0),
	'runtime': 1,
	'title': 'Blacksmith Scene',
	'tomatoes': {
		'lastUpdated': datetime(2015, 6, 28, 18, 34, 9),
		'viewer': {
			'meter': 32,
			'numReviews': 184,
			'rating': 3.0
		},
	},
	'type': 'movie',
	'year': 1893,
}
```



## Usando Python

In [1]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
warnings.filterwarnings('ignore')
mongoclient = MongoClient('localhost', 27017)
db = mongoclient.mflix
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mflix')


In [2]:
db.movies.find({}).count()

AttributeError: 'Cursor' object has no attribute 'count'

## Questões Exemplo

### [Questão Exemplo A]  Mostre um documento qualquer da coleção movies

In [3]:
result = db.movies.find({}).limit(1)

pprint( list(result) )

[
    {
        '_id': ObjectId('573a1390f29313caabcd4135'),
        'awards': {'nominations': 0, 'text': '1 win.', 'wins': 1},
        'cast': ['Charles Kayser', 'John Ott'],
        'countries': ['USA'],
        'directors': ['William K.L. Dickson'],
        'fullplot': 'A stationary camera looks at a large anvil with a blacksmith behind it and one on either side. The smith in the middle draws a heated metal rod from the fire, places it on the anvil, and all three begin a rhythmic hammering. After several blows, the metal goes back in the fire. One smith pulls out a bottle of beer, and they each take a swig. Then, out comes the glowing metal and the hammering resumes.',
        'genres': ['Short'],
        'imdb': {'id': 5, 'rating': 6.2, 'votes': 1189},
        'lastupdated': '2015-08-26 00:03:50.133000000',
        'num_mflix_comments': 1,
        'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
        'rated': 'UNRATED',
        'released': datetime.datet

### [Questão Exemplo B]  Encontre os diretores do filme "Inglourious Basterds"

In [4]:
result = db.movies.find({"title": "Inglourious Basterds"}, {"_id":0,"directors": 1})

list(result)

[{'directors': ['Quentin Tarantino', 'Eli Roth']}]

### [Questão 1] Mostre o ano de lançamento de todos os filme que começam com "One Night"

In [5]:
result = db.movies.find({'title':{'$regex':'^One Night'}},
                        {'title':1,'year':1, '_id':0}
                       )
list(result)

[{'title': 'One Night of Love', 'year': 1934},
 {'title': 'One Night Stand', 'year': 1997},
 {'title': 'One Night with the King', 'year': 2006},
 {'title': 'One Night in City', 'year': 2007}]

### [Questão 2] Quantos filmes o diretor Quentin Tarantino dirigiu?

<!--
query = { 
         "directors": "Quentin Tarantino"
}

projection = { 
               "title": 1,
               "_id": 0   
}


result = db.movies.find(query, projection).distinct("title")
list(result)
-->

In [6]:
# Contar quantos filmes o diretor Quentin Tarantino dirigiu
filmes_tarantino = db.movies.count_documents({"directors": "Quentin Tarantino"})

print(f"Quentin Tarantino dirigiu {filmes_tarantino} filmes.")


Quentin Tarantino dirigiu 14 filmes.


### [Questão 3] Quantos filmes do gênero "Drama" o diretor Quentin Tarantino dirigiu entre 1990 e 2005?


<!--
query = { 
         "directors": "Quentin Tarantino",
         "genres" : "Drama",
           "$and" : [ 
                     { "year": {"$gte":1990}},
                     { "year": {"$gte":1990}}
                    ]
}

projection = { 
               "title": 1,
               "genres":1, 
               "year": 1,
               "_id": 0
}


result = db.movies.find(query, projection)
list(result)
-->

In [7]:
# Contar quantos filmes do gênero "Drama" o diretor Quentin Tarantino dirigiu entre 1990 e 2005
filmes_drama_tarantino = db.movies.count_documents({
    "directors": "Quentin Tarantino",
    "genres": "Drama",
    "year": {"$gte": 1990, "$lte": 2005}
})

print(f"Quentin Tarantino dirigiu {filmes_drama_tarantino} filmes do gênero 'Drama' entre 1990 e 2005.")


Quentin Tarantino dirigiu 3 filmes do gênero 'Drama' entre 1990 e 2005.


### [Questão 4] Qual filme do diretor Quentin Tarantino teve mais prêmios?

Dica: Sort em Pymongo https://www.w3schools.com/python/python_mongodb_sort.asp

<!-- 
query = { 
         "directors": "Quentin Tarantino"
}

projection = { 
               "title": 1,
               "awards.wins":1,
               "_id": 0   
}


result = db.movies.find(query, projection).sort("awards.wins",-1)
list(result)
-->

In [8]:
# Encontrar o filme do Quentin Tarantino que teve mais prêmios
filme_mais_premiado = db.movies.find(
    {"directors": "Quentin Tarantino"},
    {"title": 1, "awards.wins": 1, "_id": 0}
).sort("awards.wins", -1).limit(1)

# Exibindo o resultado
for filme in filme_mais_premiado:
    print(f"O filme de Quentin Tarantino com mais prêmios é '{filme['title']}' com {filme['awards']['wins']} prêmios.")


O filme de Quentin Tarantino com mais prêmios é 'Inglourious Basterds' com 138 prêmios.


### [Questão Extra] Quais documentos (filmes) possuem mesmo nome e quantas vezes cada um se repete?

<!--

-->

In [9]:
pipeline = [
            {"$group": {"_id": "$title", "count": {"$sum": 1}}},
            {"$match": {"count":{"$gt":1}}},
            {"$sort":  {"count": -1}}
]


result = db.movies.aggregate(pipeline)
list(result)

[{'_id': 'Jane Eyre', 'count': 6},
 {'_id': 'Hamlet', 'count': 6},
 {'_id': 'The Terrorist', 'count': 6},
 {'_id': 'The Hole', 'count': 6},
 {'_id': 'The Journey', 'count': 6},
 {'_id': 'The Stranger', 'count': 5},
 {'_id': 'Les Misèrables', 'count': 5},
 {'_id': 'Madame Bovary', 'count': 5},
 {'_id': 'Peter Pan', 'count': 5},
 {'_id': 'Creature', 'count': 5},
 {'_id': 'First Love', 'count': 5},
 {'_id': 'Mad As Hell', 'count': 5},
 {'_id': 'Love', 'count': 5},
 {'_id': 'Macbeth', 'count': 5},
 {'_id': 'Alice in Wonderland', 'count': 5},
 {'_id': 'The New Land', 'count': 5},
 {'_id': 'The Emigrants', 'count': 5},
 {'_id': 'Wolf', 'count': 5},
 {'_id': 'The Collector', 'count': 4},
 {'_id': 'Harvest', 'count': 4},
 {'_id': 'Thirst', 'count': 4},
 {'_id': 'Pinocchio', 'count': 4},
 {'_id': 'Earth', 'count': 4},
 {'_id': 'Journey to the Center of the Earth', 'count': 4},
 {'_id': 'Alice', 'count': 4},
 {'_id': 'The Silence', 'count': 4},
 {'_id': 'The Hunters', 'count': 4},
 {'_id': 'The 

### [Questão Extra] Quantos filmes cada diretor participou da direção? (ignore filmes repetidos)

In [11]:
# Agrupar os filmes por diretor e contar quantos filmes cada diretor participou
filmes_por_diretor = db.movies.aggregate([
    {"$unwind": "$directors"},  
    {"$group": {"_id": "$directors", "total_filmes": {"$sum": 1}}},  
    {"$sort": {"total_filmes": -1}}  
])

# Exibir os resultados
for diretor in filmes_por_diretor:
    print(f"{diretor['_id']} dirigiu {diretor['total_filmes']} filmes.")


Woody Allen dirigiu 40 filmes.
John Ford dirigiu 35 filmes.
Takashi Miike dirigiu 34 filmes.
John Huston dirigiu 34 filmes.
Werner Herzog dirigiu 33 filmes.
Martin Scorsese dirigiu 32 filmes.
Alfred Hitchcock dirigiu 31 filmes.
Sidney Lumet dirigiu 30 filmes.
Steven Spielberg dirigiu 29 filmes.
Mario Monicelli dirigiu 29 filmes.
George Cukor dirigiu 29 filmes.
Michael Apted dirigiu 29 filmes.
Robert Altman dirigiu 28 filmes.
Steven Soderbergh dirigiu 28 filmes.
Spike Lee dirigiu 28 filmes.
Wim Wenders dirigiu 27 filmes.
Jean-Luc Godard dirigiu 27 filmes.
Clint Eastwood dirigiu 27 filmes.
Ken Loach dirigiu 27 filmes.
Johnnie To dirigiu 27 filmes.
William Wyler dirigiu 26 filmes.
Michael Winterbottom dirigiu 26 filmes.
Ingmar Bergman dirigiu 25 filmes.
Ridley Scott dirigiu 25 filmes.
Brian De Palma dirigiu 24 filmes.
Vincente Minnelli dirigiu 24 filmes.
Joseph Sargent dirigiu 24 filmes.
Claude Chabrol dirigiu 23 filmes.
Akira Kurosawa dirigiu 23 filmes.
Norman Jewison dirigiu 23 filmes.


### [Questão Extra] Qual a somatória de prêmios recebidos pelos filmes para cada diretor? (ignore filmes repetidos)

In [13]:
# Agrupar os filmes por diretor e somar os prêmios ganhos por cada diretor
premios_por_diretor = db.movies.aggregate([
    {"$unwind": "$directors"},  
    {"$group": {"_id": "$directors", "total_premios": {"$sum": "$awards.wins"}}}, 
    {"$sort": {"total_premios": -1}}  
])

# Exibir os resultados
for diretor in premios_por_diretor:
    print(f"{diretor['_id']} ganhou um total de {diretor['total_premios']} prêmios.")


Steven Spielberg ganhou um total de 699 prêmios.
Martin Scorsese ganhou um total de 587 prêmios.
Alfonso Cuarèn ganhou um total de 577 prêmios.
Peter Jackson ganhou um total de 527 prêmios.
Ethan Coen ganhou um total de 496 prêmios.
Joel Coen ganhou um total de 496 prêmios.
Christopher Nolan ganhou um total de 488 prêmios.
Quentin Tarantino ganhou um total de 445 prêmios.
Ang Lee ganhou um total de 438 prêmios.
Wes Anderson ganhou um total de 430 prêmios.
David Fincher ganhou um total de 405 prêmios.
Alejandro Gonzèlez Ièèrritu ganhou um total de 380 prêmios.
Woody Allen ganhou um total de 378 prêmios.
Steve McQueen ganhou um total de 367 prêmios.
Clint Eastwood ganhou um total de 360 prêmios.
Paul Thomas Anderson ganhou um total de 343 prêmios.
Michael Haneke ganhou um total de 332 prêmios.
Pedro Almodèvar ganhou um total de 328 prêmios.
Alexander Payne ganhou um total de 326 prêmios.
Kar Wai Wong ganhou um total de 292 prêmios.
David O. Russell ganhou um total de 287 prêmios.
James C